# Agent Implementation Documentation

## Overview

This notebook implements an AI agent that combines natural language processing capabilities with web search functionality. The implementation leverages:

- LangGraph for agent orchestration and state management
- LangChain for LLM integration and tool management
- Tavily API for web search capabilities
- OpenAI's model for natural language understanding

The agent maintains conversation state using memory checkpoints and can perform web searches to gather information in response to user queries.

## Environment Configuration

The notebook uses environment variables for secure configuration management. The `load_dotenv()` function loads API keys and other sensitive configuration from a `.env` file. This approach:

- Ensures secure handling of API keys and credentials
- Maintains separation between configuration and code
- Facilitates deployment across different environments

The `override=True` parameter ensures that existing environment variables take precedence over those in the `.env` file.

In [3]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os
  
load_dotenv(override=True)

True

## Agent Initialization

This section initializes the core components of the AI agent:

### Key Components
- `MemorySaver`: Manages conversation state and history
- `TavilySearchResults`: Implements web search functionality
- `create_react_agent`: Creates an agent that can reason and act

### Implementation Details
- The agent uses a memory checkpoint system for state persistence
- Web search is configured to return a maximum of 2 results per query
- The agent is initialized with a single tool (web search)

### Best Practices
- Tools are modular and can be extended with additional functionality
- Memory management enables context-aware conversations
- The agent architecture follows the ReAct (Reasoning and Acting) pattern

In [4]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

# Create the agent
memory = MemorySaver()
model = init_chat_model("gpt-4o-mini", model_provider="openai")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

## Basic Interaction Example

This section demonstrates basic agent interaction capabilities:

### Implementation Details
- Uses a unique thread ID for conversation tracking
- Implements streaming mode for real-time response generation
- Formats messages with clear human/agent distinction

### Key Features
- Natural language understanding and response generation
- Context-aware conversation management
- Structured message formatting for clear communication

### Technical Notes
- The `stream_mode="values"` parameter enables real-time message streaming
- Message formatting includes clear visual separation between human and agent messages
- The thread ID system enables multiple concurrent conversations

In [5]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hi Bob! It's great to meet you. How's life in San Francisco?


## Advanced Interaction with Web Search

This section demonstrates the agent's ability to combine natural language processing with web search:

### Implementation Details
- Uses the Tavily search tool to gather real-time information
- Implements token-by-token streaming for real-time response generation
- Filters messages to show only agent responses

### Key Features
- Dynamic information retrieval from the web
- Real-time response generation with token streaming
- Context-aware responses based on previous conversation

### Technical Notes
- The `stream_mode="messages"` parameter enables fine-grained control over message streaming
- Message filtering ensures only relevant agent responses are displayed
- The agent maintains context from previous interactions

In [7]:
# Stream tokens
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| is| cloudy| with| over|cast| skies|.| The| daytime| temperature| is| about| |66|°F|,| and| it| will| cool| down| to| around| |54|°F| at| night|.| 

|If| you|’d| like| more| information|,| you| can| check| out| [|this| link|](|https|://|we|athers|hog|un|.com|/weather|/|usa|/|ca|/s|an|-fr|anc|isco|/|480|/ap|ril|/|202|5|-|04|-|16|).|